# DEL 2
## Navnegeneratoren revisited
I det følgende opskalerer vi bigram-modellen, så den også kan tage "ved lære" af sine fejl - og så vi kan gøre lidt mere alvor af træningen 😊

### Spørgsmål, du skal forholde dig til i det følgende:
- Hvad er et neuralt netværk egentlig?
- I hvilken forstand kan et neuralt netværk siges "at lære"?


<b>Inden du kører</b> hver enkelt kode-celle, så prøv først at forstå, hvad koden gør!

### Importér nødvendige biblioteker

In [ ]:
import random
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

### NYE DATA: <i>list</i>  over godkendte pigenavne (jf. dansk Navnelov)

In [ ]:
words = open('data/navne.txt', 'r').read().splitlines()
words[:8]

### Lav en <i>dictionary</i>: bogstav &rarr; tal (index) - og omvendt

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)
print("")
print(f"Antal tegn i vores 'alfabet': {len(itos)}")

## Vi opskalerer vores håndskrevne model:

### Byg et datasæt
- Overvej, hvorfor tensorerne får de dimensioner, de gør nedenfor?
- Prøv at lege med værdien for <code>block_size</code> - hvilken betydning har det for modellen?

In [ ]:
block_size = 3

def build_dataset(words):  
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]

  X = torch.tensor(X) # vores eksempler
  Y = torch.tensor(Y) # deres 'labels'
  print(X.shape, Y.shape)
  return X, Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

### Hvis vi bare plottede vores alfabet (fra datasættet) fuldstændig tilfældigt, hvordan ville det så se ud?

In [ ]:
C = torch.randn((109, 2))
plt.figure(figsize=(8,8))
plt.scatter(C[:,0].data, C[:,1].data, s=200)
for i in range(C.shape[0]):
    plt.text(C[i,0].item(), C[i,1].item(), itos[i], ha="center", va="center", color='white')
plt.grid('minor')

### Indlæs modellens parametre, dvs. C, vægtene (W1, W2) samt bias (b1, b2) - tilfældigt

In [ ]:
C = torch.randn((109, 10))
W1 = torch.randn((30, 200))
b1 = torch.randn(200)
W2 = torch.randn((200, 109)) # Lag med 200 neuroner
b2 = torch.randn(109)
parameters = [C, W1, b1, W2, b2]

In [ ]:
num_parameters = sum(p.nelement() for p in parameters)
print(f'Antal parametre for vores model: {num_parameters}')

In [ ]:
for p in parameters:
  p.requires_grad = True # vi vil gerne beregne gradienter/hældning

#### Modellens <i>learning rate</i>

In [ ]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [ ]:
lri = []
lossi = []
stepi = []

### Træn modellen med 100.000 kørsler
- Forsøg med lidt forskellige kørsler og se, hvad det gør ved modellens ydeevne?
- Hvad gør "forward" og "backward" pass egentlig?

In [ ]:
for i in range(100000):
  
  ix = torch.randint(0, Xtr.shape[0], (32,))
  
  # "Forward pass"
  emb = C[Xtr[ix]]
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, Ytr[ix])

  # "Backward pass"
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # Opdatér model
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad

  # Følg udviklingen
  stepi.append(i)
  lossi.append(loss.log10().item())

In [ ]:
print(f'Hvad siger vores loss: {loss}')

#### Evaluér modellen

In [ ]:
plt.plot(stepi, lossi)

### Hvordan ser det ud, nu hvor modellen forhåbentlig har fundet lidt struktur i dataene?

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(C[:,0].data, C[:,1].data, s=200)
for i in range(C.shape[0]):
    plt.text(C[i,0].item(), C[i,1].item(), itos[i], ha="center", va="center", color='white')
plt.grid('minor')

In [ ]:
context = [0] * block_size
C[torch.tensor([context])].shape

### Lad os "sample" fra modellen

In [ ]:
samples = 20

for _ in range(samples): # gentag bare nedenstående x-antal gange; vi skal alligevel ikke bruge index-variablen
    
    out = []
    context = [0] * block_size
    while True:
      emb = C[torch.tensor([context])]
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

### I øvrigt - vedrørende matrixmultiplikation &#129300;
Som sagt kan modellen ikke arbejde med tekst i helt almindelig forstand, men der er behov for at omsætte "teksten" til tal - de såkaldte <i>embeddings</i>. Her er der forskellige strategier, fra den helt naive - som ovenfor: arrangér tegn i en <i>dictionary</i>, hvor hvert index så repræsenterer det pågældende bogstav, over til mere avancerede, som forsøger at få selve konteksten for et ord med i dets embedding, fx ved brug af trigonometriske funktioner (som det er tilfældet med GPT).

In [ ]:
C = torch.randn((109, 3)) # vi forsøger at nedskalere 109 "kategorier" (vores alfabet) til 3 dimensioner

In [ ]:
stoi['e'] # omsæt 'e' til et heltal (jf. tabellen ovenfor)

In [ ]:
C[34] # et udmærket bud på en embedding af bogstavet 'e'

#### Men er det anderledes end at have lavet en one-hot-encoding og så matrix multiplicere med C? Lad os prøve ...

In [ ]:
F.one_hot(torch.tensor(34), num_classes=109)

In [ ]:
F.one_hot(torch.tensor(34), num_classes=109).float() @ C